# Supplementary: Personalized PageRank

In this notebook we apply pagerank to our most significant GWAS hits using the power method supplied in networkx. We also try to build an iterative algorithm because networkx.pagerank was too computationally intensive to run on the whole network on my laptop. Thus, initially the networkx implementation is run on the top ~4000 most significant genes, but the iterative implementation used the entire PCNet network. We also tried the networkx implementation on the entire network, which performed approximately 10% better than using the minimal network. 

-log p-values are mapped to genes based on whether the snp lies in the gene body, promoter, or in an enhancer that likely regulates the transcription of a gene. See Remapping SNPs Part II for more information as to how this was done. The smallest p-value for each gene is used for the analysis, and the normalized personalization vector consists of the normalized -log p-values. 0.85 is the most used decay factor (alpha) in PPR calculations, so we elected to use that value here. 

Note: After trying to remap the p-values to the genehancer and enhancer atlas databases, they were found to perform poorly compared to mapping genes according to proximity to snps (within 10 kb). 

The iterative equation for PageRank is found below:

x = alpha*P*x + (1-alpha)*b

The first term in this equation describes a random walk, where P is the transition matrix (transpose of the network) and alpha is the decay factor. The second term defines the personalization, where b is the personalization vector. 

After implementing PPR, we validate overlap between the top 260 genes and those identified in the CTD database, as well as those identified in the schizophrenia TWAS study.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import time
import scipy
import copy

In [2]:
gene_stats = pd.read_csv("snp_remapped.csv") #enhancer_atlas mapping
gene_stats['-logP'] = -np.log10(gene_stats['pval'])+np.log10(0.005)
gene_stats.head()

,Unnamed: 0,snpid,gene,chr,bp,pval,-logP
0,0,rs6494623,AAGAB,chr15,65118060,0.001136,0.643752
1,1,rs9395010,AARS2,chr6,44453984,0.000047,2.023902
2,2,rs1317763,ABCA12,chr2,216158900,0.002019,0.393793
3,3,rs1963367,ABCB6,chr2,220582392,0.001261,0.598117
4,4,rs4401068,ABCC11,chr16,45354669,0.002305,0.336337


In [3]:
from ndex.networkn import NdexGraph
import pickle
import operator

def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
#Import PCNet

dG=NdexGraph(server='http://www.ndexbio.org', uuid= 'f93f402c-86d4-11e7-a10d-0ac135e8bacf')
G=nx.Graph(dG)
G=nx.relabel_nodes(G,nx.get_node_attributes(dG,'name'),copy=True)
save_object(G, 'G.pkl')

In [4]:
with open('G.pkl', 'rb') as input:
    G = pickle.load(input)

In [5]:
#Reduce size of network to allow calculation on laptop

G_min = G.subgraph(gene_stats['gene']) #gene stats only contains genes with p values <0.005
len(G_min)

1996

In [6]:
#Make personalization vector

b = {} #dictionary of normalized -log p-values

for n, d in enumerate(G_min.nodes): 
    b.update({d:(-np.log10(gene_stats['pval'][n])+np.log10(0.005))/(-np.log10(gene_stats['pval'][0])+np.log10(0.005))})

len(b)

1996

In [28]:
#First implement analytical solution in networkx

ppr_schiz = nx.pagerank(G_min, alpha = 0.85, personalization=b)
ppr_schiz = sorted(ppr_schiz.items(), key=operator.itemgetter(1), reverse = True)
ppr_schiz = pd.DataFrame(ppr_schiz)
ppr_schiz.head()

,0,1
0,APP,0.004953
1,MYC,0.004066
2,HSPA8,0.003589
3,SP1,0.003339
4,POLR2B,0.002835


In [29]:
#export list

ppr_schiz.iloc[0:260,0].to_csv('ppr_schiz_networkx_enhanceratlas.csv')

In [7]:
#analytical solution, functions for random walk adapted from Idekar Lab GITHUB

def normalize_network(network, symmetric_norm=False):
    adj_mat = nx.adjacency_matrix(network)
    adj_array = np.array(adj_mat.todense())
    if symmetric_norm:
        D = np.diag(1/np.sqrt(sum(adj_array)))
        adj_array_norm = np.dot(np.dot(D, adj_array), D)
    else:
        degree_norm_array = np.diag(1/sum(adj_array).astype(float))
        sparse_degree_norm_array = scipy.sparse.csr_matrix(degree_norm_array)
        adj_array_norm = sparse_degree_norm_array.dot(adj_mat).toarray()
    return adj_array_norm

# PPR, iterative 
def ppr_iterative (alpha, binary_mat,adj_array_norm): #binary mat is personalization
    t=0
    n=len(adj_array_norm) #number of nodes
    term0=np.zeros((1,n)) #initial values
    term1=term0 #updates each time
    while True:
        #x = alpha*P*x + (1-alpha)*b
        term2 = np.dot((1-alpha)*term1,adj_array_norm) + (1-alpha)*binary_mat #restart probability is related to personalization
        t +=1
        dist = np.linalg.norm(term2-term1)
        if dist < 0.000001:
            print('iterate times t:',t)
            return term2
            break
        else:
            term1=term2 
        if t==21:
            print('time out at t = 15')
            break

# RWR, iterative
def rwr_iterative (alpha, binary_mat,adj_array_norm): #binary mat is personalization
    t=0
    n=len(adj_array_norm) #number of nodes
    term0=binary_mat #initial values
    term1=term0 #updates each time
    while True:
        #x = alpha*P*x + (1-alpha)*b
        term2 = np.dot((1-alpha)*term1,adj_array_norm) + alpha*term0
        t +=1
        dist = np.linalg.norm(term2-term1)
        if dist < 0.000001:
            print('iterate times t:',t)
            return term2
            break
        else:
            term1=term2  
        if t==21:
            print('time out at t = 15')
            break
            
# create personalization vector function
def create_personalization_mat(network, b):
    nodes=network.nodes()
    n=len(nodes)
    nodes_idx_map=dict(list(zip(nodes,range(n))))
    b_mat=np.zeros((1,n))
    for i in b.keys():
         if i in nodes:
                b_mat[0,nodes_idx_map[i]]=b[i]
    return b_mat

In [32]:
#Make new personalization vector (unchanged)

b = {} #dictionary of normalized -log p-values

for n, d in enumerate(G_min.nodes): 
    b.update({d:(-np.log10(gene_stats['pval'][n])+np.log10(0.005))/(-np.log10(gene_stats['pval'][0])+np.log10(0.005))})

len(b)

1996

In [8]:
# normalize the network of G 
G_adj_array_norm= normalize_network(G)

#set alpha
alpha = 0.85

#personalization matrix
b_mat=create_personalization_mat(G, b)

In [ ]:
#perform PPR

ppr_schiz_it = ppr_iterative(alpha, b_mat, G_adj_array_norm)

In [36]:
#get ranked list of top 260 genes and save as csv

nodes_ppr=dict(list(zip(G.nodes,ppr_schiz_it[0])))
ppr_genes = sorted(nodes_ppr, key=nodes_ppr.get, reverse=True)
ppr_genes = ppr_genes[0:260]
pd.DataFrame(ppr_genes).iloc[0:260,:].to_csv('ppr_schiz_iterative_enhanceratlas.csv')

In [37]:
#Compare networkx ppr ranking to CTD database

curated_list = pd.read_csv('CTD_D012559_genes_20180607152155.csv')
curated_list = curated_list.iloc[0:260,:].loc[:,['Gene Symbol']]
curated_list

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [38]:
len(intersection(ppr_schiz.iloc[0:260,0].as_matrix(),curated_list.as_matrix()))/260 

0.023076923076923078

In [39]:
#Compare iterative solution to CTD database

len(intersection(pd.DataFrame(ppr_genes).iloc[0:260,:].as_matrix(),curated_list.as_matrix()))/260 

0.011538461538461539

In [56]:
#In this case it looks like the networkx implementation works better

#Let's look at the original gene list (provided to all of the class) and perform the networkx implementation

gene_level_summary_stats = pd.read_csv("C:\\Users\\magne\\Documents\\Classes\\BNFO286\\final_project\\gene_level_summary_stats_pmid_25056061.txt", sep = '\t')
gene_p = gene_level_summary_stats.loc[:,['Gene','TopSNP P-Value']]
gene_p_min = gene_p.loc[gene_p['TopSNP P-Value'] < 0.05]
p_list = gene_p_min.values.T.tolist()
G_min = G.subgraph(p_list[0])
d_list = {}
for n, d in enumerate(p_list[0]): #dictionary of inverse, normalized p-values (to 1)
    d_list.update({d:(-np.log10(p_list[1][n])+np.log10(0.005))/(-np.log10(p_list[1][0])+np.log10(0.005))})
pr_schiz_personalization = nx.pagerank(G_min, alpha = 0.85, personalization=d_list, max_iter = 10)
schiz_per = pd.DataFrame(sorted(pr_schiz_personalization.items(), key=operator.itemgetter(1), reverse = True))
schiz_per = schiz_per.iloc[:,0].iloc[0:260]
len(intersection(schiz_per.as_matrix(),curated_list.as_matrix()))/260

0.08461538461538462

8% overlap between the database and the results of the network propagation indicates an ~8 fold enrichment of schizophrenia genes over random. 

It looks like PPR performs best with the original gene set. This increase in performance can perhaps be attributed to not mapping less relevant genes to the gene list. The original gene set is also twice as large as the remapped gene list because the the p-value cut-off was increased to 0.05.

In [23]:
# Try RWR-iterative again for comparison

b = {} #dictionary of normalized -log p-values

for n, d in enumerate(G.nodes): 
    b.update({d:(-np.log10(gene_stats['pval'][n])+np.log10(0.005))})

# normalize network G 
G_adj_array_norm= normalize_network(G)

#set alpha
alpha = 0.44

#heat matrix

b_mat=create_personalization_mat(G, b)

#perform RWR

rwr_schiz_it = rwr_iterative(alpha, b_mat, G_adj_array_norm)

nodes_rwr=dict(list(zip(G.nodes,rwr_schiz_it[0])))
rwr_genes = sorted(nodes_rwr, key=nodes_rwr.get, reverse=True)
rwr_genes = rwr_genes[0:260]
pd.DataFrame(rwr_genes).iloc[0:260,:].to_csv('rwr_schiz_iterative.csv')

#Compute intersection

len(intersection(pd.DataFrame(ppr_genes).iloc[0:260,:].as_matrix(),curated_list.as_matrix()))/260 

C:\Users\magne\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


iterate times t: 20


0.011538461538461539

The RWR performs slightly worse in this case than the PPR

Things to try:

Since multiple genes per snps in enhancer within genehancer database, there may be many false positives which are skewing our result. One thing to do might be to remove p-values from gene that have only one significant p-value.

We can also try mapping according to TSS's that lie within 10 kb of snp.

Conclusion: genehancer likely contains too many false positives to be useful for this GWAS. 


